In [3]:
import numpy as np
import pandas as pd
import opendatasets as od

od.download(
    "https://www.kaggle.com/datasets/nelgiriyewithana/credit-card-fraud-detection-dataset-2023")


ModuleNotFoundError: No module named 'opendatasets'